# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
3. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

# Function to clean the notes 
def clean_notes(team): 
    # Search for opening bracket in the name followed by 
    # any characters repeated any number of times 
    if re.search('\[.*', team): 
  
        # Extract the position of beginning of pattern 
        pos = re.search('\[.*', team).start() 
  
        # return the cleaned name 
        return team[:pos] 
  
    else: 
        # if clean up needed return the same name 
        return team 

def clean_teams(team):
    splited = team.split(" ")[-1]
    if splited.endswith("*"):
        return splited[:-1]
    else:
        return splited

def split_it(x):
    return re.sub(r"([A-Z])", r" \1", x).split()


    
nhl_df=pd.read_csv("assets/nhl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities = cities.iloc[:-1,[0,3,5,6,7,8]]
cities = cities.loc[:,["Metropolitan area", "Population (2016 est.)[8]", "NHL"]]
cities = cities[(cities["NHL"] != "—") ] 
cities = cities[~cities["NHL"].str.startswith("[")]
cities["NHL"] = cities["NHL"].apply(clean_notes)
cities.rename(columns = {"Population (2016 est.)[8]":"Population"}, inplace = True) 
cities.reset_index(inplace=True, drop=True)

s = cities.loc[~cities["NHL"].str.contains("\s"), "NHL"]
cities = cities.merge(s.str.extractall("([A-Z][a-z]+)").droplevel(1), how="left",
                left_index=True, right_index=True).ffill(1)
cities.rename(columns = {0:"team"}, inplace = True) 
cities.reset_index(inplace=True, drop=True)
del cities["NHL"]


cities['team'] = cities['team'].replace("Red Wings", "Wings")
cities['team'] = cities['team'].replace("Maple Leafs", "Leafs")
cities['team'] = cities['team'].replace("Blue Jackets", "Jackets")
cities['team'] = cities['team'].replace("Golden Knights", "Knights")

print(cities)

nhl_df = nhl_df[~nhl_df["team"].str.endswith("Division") & (nhl_df["year"] == 2018)]
nhl_df = nhl_df.loc[:, ["team", "W", "L"]]
nhl_df.reset_index(inplace=True, drop=True)
nhl_df["team"] = nhl_df["team"].apply(clean_teams)
print(nhl_df)

# print(nhl_df["team"])

print(len(nhl_df))  
new_df = pd.merge(cities, nhl_df,  how='right', on="team")
new_df["wl_rate"] = new_df["W"].astype(int) / (new_df["W"].astype(int) + new_df["L"].astype(int))

res = new_df.groupby(['Metropolitan area', 'Population']).mean().reset_index()

# print(list(res["Population"]))

def nhl_correlation(): 
    # YOUR CODE HERE
    # raise NotImplementedError()
    
    population_by_region = list(res['Population']) # pass in metropolitan area population from cities
    win_loss_by_region = list(res['wl_rate']) # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]
    population_by_region = [int(i) for i in population_by_region]

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]
nhl_correlation()

         Metropolitan area Population        team
0            New York City   20153634     Rangers
1            New York City   20153634   Islanders
2            New York City   20153634      Devils
3              Los Angeles   13310447       Kings
4              Los Angeles   13310447       Ducks
5   San Francisco Bay Area    6657982      Sharks
6                  Chicago    9512999  Blackhawks
7        Dallas–Fort Worth    7233323       Stars
8         Washington, D.C.    6131977    Capitals
9             Philadelphia    6070500      Flyers
10                  Boston    4794447      Bruins
11  Minneapolis–Saint Paul    3551036        Wild
12                  Denver    2853077   Avalanche
13   Miami–Fort Lauderdale    6066387    Panthers
14                 Phoenix    4661537     Coyotes
15                 Detroit    4297617       Wings
16                 Toronto    5928040       Leafs
17          Tampa Bay Area    3032171   Lightning
18              Pittsburgh    2342299    Penguins


0.012486162921209907

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [28]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def clean_notes(team): 
    if re.search('\[.*', team): 
  
        # Extract the position of beginning of pattern 
        pos = re.search('\[.*', team).start() 
  
        # return the cleaned name 
        return team[:pos] 
  
    else: 
        # if clean up needed return the same name 
        return team 

def clean_teams2(team):
    team = team.replace("\xa0", " ")
    team = team.replace("*", "")
    splited = team.split(" ")
    return splited[-2]

def split_it(x):
    return re.sub(r"([A-Z])", r" \1", x).split()

    
nba_df=pd.read_csv("assets/nba.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities = cities.iloc[:-1,[0,3,5,6,7,8]]
cities = cities.loc[:,["Metropolitan area", "Population (2016 est.)[8]", "NBA"]]
cities = cities[(cities["NBA"] != "—") ] 
cities = cities[~cities["NBA"].str.startswith("[")]
cities["NBA"] = cities["NBA"].apply(clean_notes)
cities.rename(columns = {"Population (2016 est.)[8]":"Population"}, inplace = True) 
cities.reset_index(inplace=True, drop=True)

s = cities.loc[~cities["NBA"].str.contains("\s"), "NBA"]
cities = cities.merge(s.str.extractall("([A-Z][a-z]+)").droplevel(1), how="left",
                left_index=True, right_index=True).ffill(1)
cities.rename(columns = {0:"team"}, inplace = True) 
cities.reset_index(inplace=True, drop=True)
del cities["NBA"]

cities['team'] = cities['team'].replace("Trail Blazers", "Blazers")

nba_df = nba_df[nba_df["year"] == 2018]
nba_df = nba_df.loc[:, ["team", "W", "L"]]
nba_df.reset_index(inplace=True, drop=True)
nba_df["team"] = nba_df["team"].apply(clean_teams2)

# print(nhl_df["team"])
 
new_df2 = pd.merge(cities, nba_df,  how='right', on="team")
new_df2["wl_rate"] = new_df2["W"].astype(int) / (new_df2["W"].astype(int) + new_df2["L"].astype(int))

res = new_df2.groupby(['Metropolitan area', 'Population']).mean().reset_index()

def nba_correlation():
    # YOUR CODE HERE
    
    population_by_region = list(res['Population']) # pass in metropolitan area population from cities
    win_loss_by_region = list(res['wl_rate']) # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]
    
    population_by_region = [int(i) for i in population_by_region]
    
    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]
nba_correlation()

-0.17657160252844617

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [74]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def clean_notes(team): 
    if re.search('\[.*', team): 
  
        # Extract the position of beginning of pattern 
        pos = re.search('\[.*', team).start() 
  
        # return the cleaned name 
        return team[:pos] 
  
    else: 
        # if clean up needed return the same name 
        return team 

def clean_teams3(team):
    splited = team.split(" ")
    return splited[-1]

def split_it(x):
    return re.sub(r"([A-Z])", r" \1", x).split()

    
mlb_df=pd.read_csv("assets/mlb.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities = cities.iloc[:-1,[0,3,5,6,7,8]]
cities = cities.loc[:,["Metropolitan area", "Population (2016 est.)[8]", "MLB"]]
cities = cities[(cities["MLB"] != "—") ] 
cities = cities[~cities["MLB"].str.startswith("[")]
cities["MLB"] = cities["MLB"].apply(clean_notes)
cities.rename(columns = {"Population (2016 est.)[8]":"Population"}, inplace = True) 
cities.reset_index(inplace=True, drop=True)

s = cities.loc[~cities["MLB"].str.contains("\s"), "MLB"]
cities = cities.merge(s.str.extractall("([A-Z][a-z]+)").droplevel(1), how="left",
                left_index=True, right_index=True).ffill(1)
cities.rename(columns = {0:"team"}, inplace = True) 
cities.reset_index(inplace=True, drop=True)

cities['team'] = cities['team'].replace("White Sox", "Wsox")
cities['team'] = cities['team'].replace("Red Sox", "Rsox")
cities['team'] = cities['team'].replace("Blue Jays", "Jays")

s = cities.loc[cities["team"].str.contains("\s"), "team"]
cities = cities.merge(s.str.extractall("([A-Za-z]+)").droplevel(1), how="left",
                left_index=True, right_index=True).ffill(1)
cities.rename(columns = {0:"team_final"}, inplace = True) 
cities.reset_index(inplace=True, drop=True)

del cities["team"]
del cities["MLB"]

cities['team'] = cities['team_final'].replace("Sox", "Wsox")
cities['team'] = cities['team_final'].replace("CubsWhite", "Cubs")
#print(cities)

mlb_df = mlb_df[mlb_df["year"] == 2018]
mlb_df = mlb_df.loc[:, ["team", "W", "L"]]
mlb_df.reset_index(inplace=True, drop=True)
mlb_df["team"] = mlb_df["team"].apply(clean_teams3)
mlb_df["team"][0] = "Rsox"
#print(mlb_df)

 
new_df3 = pd.merge(cities, mlb_df,  how='right', on="team")
new_df3["wl_rate"] = new_df3["W"].astype(float) / (new_df3["W"].astype(float) + new_df3["L"].astype(float))

res = new_df3.groupby(['Metropolitan area', 'Population']).mean().reset_index()
#print(res)

def mlb_correlation(): 
    # YOUR CODE HERE
    # raise NotImplementedError()
    
    population_by_region = list(res['Population']) # pass in metropolitan area population from cities
    win_loss_by_region = list(res['wl_rate']) # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]
    
    population_by_region = [int(i) for i in population_by_region]

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]
mlb_correlation()

         Metropolitan area Population    team_final          team
0            New York City   20153634       Yankees       Yankees
1            New York City   20153634          Mets          Mets
2              Los Angeles   13310447       Dodgers       Dodgers
3              Los Angeles   13310447        Angels        Angels
4   San Francisco Bay Area    6657982        Giants        Giants
5   San Francisco Bay Area    6657982     Athletics     Athletics
6                  Chicago    9512999     CubsWhite          Cubs
7                  Chicago    9512999           Sox           Sox
8        Dallas–Fort Worth    7233323       Rangers       Rangers
9         Washington, D.C.    6131977     Nationals     Nationals
10            Philadelphia    6070500      Phillies      Phillies
11                  Boston    4794447          Rsox          Rsox
12  Minneapolis–Saint Paul    3551036         Twins         Twins
13                  Denver    2853077       Rockies       Rockies
14   Miami

0.15027698302669307

## Question 4 (completed for you)
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [23]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def clean_notes(team): 
    if re.search('\[.*', team): 
  
        # Extract the position of beginning of pattern 
        pos = re.search('\[.*', team).start() 
  
        # return the cleaned name 
        return team[:pos] 
  
    else: 
        # if clean up needed return the same name 
        return team 

def clean_teams(team):
    splited = team.split(" ")[-1]
    if splited.endswith("*") or splited.endswith("+"):
        return splited[:-1]
    else:
        return splited

def split_it(x):
    return re.sub(r"([A-Z])", r" \1", x).split()

nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

cities = cities.loc[:,["Metropolitan area", "Population (2016 est.)[8]", "NFL"]]
cities = cities[~cities["NFL"].str.startswith("—")] 

cities = cities[~cities["NFL"].str.startswith("[")]
cities["NFL"] = cities["NFL"].apply(clean_notes)
cities.rename(columns = {"Population (2016 est.)[8]":"Population"}, inplace = True) 
cities.rename(columns = {"NFL":"team"}, inplace = True) 
cities.reset_index(inplace=True, drop=True)

s = cities.loc[~cities["team"].str.contains("\s"), "team"]
cities = cities.merge(s.str.extractall("([A-Z][a-z]+)").droplevel(1), how="left",
                left_index=True, right_index=True).ffill(1)
cities.rename(columns = {0:"team_final"}, inplace = True) 
cities.reset_index(inplace=True, drop=True)

#print(cities)

nfl_df = nfl_df[nfl_df["year"] == 2018]
nfl_df = nfl_df.loc[:, ["team", "W", "L"]]
nfl_df = nfl_df[~nfl_df["team"].str.startswith("AFC")]
nfl_df = nfl_df[~nfl_df["team"].str.startswith("NFC")]
nfl_df.reset_index(inplace=True, drop=True)
nfl_df["team_final"] = nfl_df["team"].apply(clean_teams)

#print(nfl_df)

new_df4 = pd.merge(cities, nfl_df,  how='right', on="team_final")
new_df4["wl_rate"] = new_df4["W"].astype(int) / (new_df4["W"].astype(int) + new_df4["L"].astype(int))

res = new_df4.groupby(['Metropolitan area', 'Population']).mean().reset_index()

#print(res)

def nfl_correlation(): 
    # YOUR CODE HERE
    
    
    population_by_region = list(res['Population']) # pass in metropolitan area population from cities
    win_loss_by_region = list(res['wl_rate']) # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]
    
    population_by_region = [int(i) for i in population_by_region]

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

## Question 5 (completed for you)
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
nhl_df=pd.read_csv("assets/nhl.csv")
nba_df=pd.read_csv("assets/nba.csv")
nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def sports_team_performance():
    # YOUR CODE HERE
    raise NotImplementedError()
    
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values